In [4]:
!pip install selenium

In [1]:
import csv
import sys
from datetime import date
import pandas as pd
import selenium.webdriver
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from dateutil.relativedelta import relativedelta

In [2]:
#for exception
import linecache
import sys

def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
def scrap(origin,destin,trDate):
    global none
    # User defined variables for data retreival
    #origin = "BOM" 				# Origin airport code
    #destin = "DEL" 				# Destination airport code
    #trDate = ('30'+'/'+'09'+'/2021')			# Date as 1st command line argument.
    #print(sys.argv)

    """ The following is the Base Url for fetching data from MakeMyTrip Website.
        This URL appears in the search bar after origin, destination and date inputs on the landing page.
        Thus, this URL can be changed based on User Inputs and required data can be fetched.
    """
    baseDataUrl = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ destin +"-"+ trDate +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E"

    try:
        driver = selenium.webdriver.Chrome('chromedriver.exe') # Chrome driver is being used.
        print ("Requesting URL: " + baseDataUrl)

        driver.get(baseDataUrl)  			 # URL requested in browser.
        print ("Webpage found ...")

        element_xpath = '//*[@id="left-side--wrapper"]/div[2]' # First box with relevant flight data.

        # Wait until the first box with relevant flight data appears on Screen
        element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

        # Scroll the page till bottom to get full data available in the DOM.
        print ("Scrolling document upto bottom ...")
        for j in range(1, 100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

        print("Closing Chrome ...") # No more usage needed.
        driver.quit() 				# Browser Closed.

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
        
        # Extract the required tags 
        spanFlightName = soupBody.find_all("span", 'boldFont blackText airlineName')		# Tags with Flight Name
        print("spanFlightName  = " + str(len(spanFlightName)))

        Time = soupBody.find_all("p", {"class":"blackText fontSize18 blackFont appendBottom2 makeFlex hrtlCenter"})			# Tags with Departure Time
        print("Time Count in Scraping  = " + str(len(Time)))

        City = soupBody.findAll("p", {"class": "darkText"})				# Tags with Departure City
        print("City Count in Scraping  = " + str(len(City)))

        spanFlightCost = soupBody.findAll("p", {"class": "blackText fontSize18 blackFont white-space-no-wrap"})			# Tags with Flight Cost
        print("spanFlightCost Count in Scraping  = " + str(len(spanFlightCost)))

        pFlightStops = soupBody.findAll("p", {"class": "flightsLayoverInfo"})			# Tags with flight stops
        print("pFlightStops Count in Scraping  = " + str(len(pFlightStops)))
        
        transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops)



   
    except :
        PrintException()
        print(trDate," scraping complete")
        
        none =True 
    # EOF
    # ----------------------------------------------------------------------------------------------------------
    #print("Records\nFlight Name: "+ str(len(spanFlightName)) + "\nFlightCode: "+ str(len(pFlightCode)) + "\nDept Time: "+ str(len(divDeptTime)) + "\nDept City: "+ str(len(pDeptCity)) + "\nFlight Duration: "+ str(len(pFlightDuration)) + "\nArrival Time: "+ str(len(pArrivalTime)) + "\nArrival City: "+ str(len(pArrivalCity)) + "\nFlight Cost: "+ str(len(spanFlightCost)))
    #print(flightsData)
    #print(outputFile)
    

In [4]:
dates=(pd.date_range(start='15/10/2021', end='15/11/2021')) #dates 

New_date=[]
for i,date in enumerate(dates):
    day=str(date.day).zfill(2)   # 1 --->01,2--->02 conversion
    month=str(date.month).zfill(2)
    year=str(date.year)
    
    date=day+'/'+month+'/'+year  #formating date
    New_date.append(date)
print(New_date)    

['15/10/2021', '16/10/2021', '17/10/2021', '18/10/2021', '19/10/2021', '20/10/2021', '21/10/2021', '22/10/2021', '23/10/2021', '24/10/2021', '25/10/2021', '26/10/2021', '27/10/2021', '28/10/2021', '29/10/2021', '30/10/2021', '31/10/2021', '01/11/2021', '02/11/2021', '03/11/2021', '04/11/2021', '05/11/2021', '06/11/2021', '07/11/2021', '08/11/2021', '09/11/2021', '10/11/2021', '11/11/2021', '12/11/2021', '13/11/2021', '14/11/2021', '15/11/2021']


In [5]:
# various list container
Airline=[]
Journey_date=[]
From=[]
To=[]
Price=[]
Dtime=[]
Atime=[]
Stops=[]

flight_data = pd.DataFrame(np.nan, index=[0], columns=['Airline', 'Journey_date','From','To','Dtime','Atime','Stops','Price'])
flight_data.to_csv('Flight.csv')
#transforming scraped data 
def transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops):
    
    Airline.clear()
    Journey_date.clear()
    Price.clear()
    Stops.clear()
    From.clear()
    Dtime.clear()
    To.clear()
    Atime.clear()
    
    global flight_data
    print("Transaform Function Running.... ")
    for i in range(0,len(spanFlightName)):
        
        Airline.append(spanFlightName[i].string)
        Journey_date.append(trDate)
        Price.append(spanFlightCost[i].string)
        Stops.append(pFlightStops[i].string)
        
      
    for j in range(0,len(Time)):
        
        if(j%2==0):
            
            From.append(City[j].string)
            Dtime.append(Time[j].string)
        else:
            
            To.append(City[j].string)
            Atime.append(Time[j].string) 

            
            
    temp = {'Airline':Airline,
            'Journey_date':Journey_date,
            'From':From,
            'To':To,
            'Dtime':Dtime,
            'Atime':Atime,
            'Stops':Stops,
            'Price':Price}
    df=pd.DataFrame(temp)
    
    df.to_csv('Flight.csv', mode='a', header=False)
        
    flight_data = pd.concat([flight_data,df])
    flight_data=flight_data.reset_index(drop=True)
    display(flight_data)
    print('Lenght : ',len(flight_data))

In [ ]:
## applying various state combination and extrating  1 month flight data

list_of_state=['RPR','GOI','AMD','IXC','IXR','IXW','VTZ','TIR','KUU','BLR','COK','IDR','PNQ','BOM','DEL','IMF','SHL','AJL','DMU','BBI','LUH','JAI','MAA','LKU','VNS','DED','CCU']
# 
for i in range(0,len(list_of_state)):
    orgin=list_of_state[i]
    for_to=list_of_state
    
    
    
    print("--------------")
    for j in range(0,len(for_to)):
        destin=for_to[j]
        none = False
        print("origin = "+orgin)
        print("destin = "+destin+'\n')
        
        if orgin!=destin:
            for i,date in enumerate(New_date) :
                if none !=True:
                    scrap(orgin,destin,date)
                    print(" ")
                else:
                    break
                
        else: 
            
            print( "-----Skiped-----")
            print(" ")
    
        

--------------
origin = PAT
destin = PAT

-----Skiped-----
 
origin = PAT
destin = jknaknnaknja

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=PAT-jknaknnaknja-15/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
EXCEPTION IN (<ipython-input-3-e63b93705f05>, LINE 25 "element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))"): Message: 

15/10/2021  scraping complete
 
origin = PAT
destin = MAA

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=PAT-MAA-15/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 14
Time Count in Scraping  = 28
City Count in Scraping  = 30
spanFlightCost Count in Scraping  = 14
pFlightStops Count in Scraping  = 14
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Go First,15/10/2021,Patna,Chennai,16:55,23:15,1 stop via Mumbai,"₹ 7,511"
2,IndiGo,15/10/2021,Patna,Chennai,08:40,16:10,1 stop via Bengaluru,"₹ 7,512"
3,IndiGo,15/10/2021,Patna,Chennai,11:20,17:30,1 stop via Mumbai,"₹ 7,512"
4,IndiGo,15/10/2021,Patna,Chennai,11:35,17:55,1 stop via Hyderabad,"₹ 7,512"
5,IndiGo,15/10/2021,Patna,Chennai,11:50,19:35,1 stop via Bengaluru,"₹ 7,512"
6,IndiGo,15/10/2021,Patna,Chennai,13:20,20:25,1 stop via Kolkata,"₹ 7,512"
7,Spicejet,15/10/2021,Patna,Chennai,15:05,21:25,1 stop via Bengaluru,"₹ 7,512"
8,IndiGo,15/10/2021,Patna,Chennai,13:00,21:50,1 stop via Ranchi,"₹ 7,512"
9,IndiGo,15/10/2021,Patna,Chennai,16:30,23:35,1 stop via New Delhi,"₹ 7,512"


Lenght :  15
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=PAT-MAA-16/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 13
Time Count in Scraping  = 26
City Count in Scraping  = 26
spanFlightCost Count in Scraping  = 13
pFlightStops Count in Scraping  = 13
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Go First,15/10/2021,Patna,Chennai,16:55,23:15,1 stop via Mumbai,"₹ 7,511"
2,IndiGo,15/10/2021,Patna,Chennai,08:40,16:10,1 stop via Bengaluru,"₹ 7,512"
3,IndiGo,15/10/2021,Patna,Chennai,11:20,17:30,1 stop via Mumbai,"₹ 7,512"
4,IndiGo,15/10/2021,Patna,Chennai,11:35,17:55,1 stop via Hyderabad,"₹ 7,512"
5,IndiGo,15/10/2021,Patna,Chennai,11:50,19:35,1 stop via Bengaluru,"₹ 7,512"
6,IndiGo,15/10/2021,Patna,Chennai,13:20,20:25,1 stop via Kolkata,"₹ 7,512"
7,Spicejet,15/10/2021,Patna,Chennai,15:05,21:25,1 stop via Bengaluru,"₹ 7,512"
8,IndiGo,15/10/2021,Patna,Chennai,13:00,21:50,1 stop via Ranchi,"₹ 7,512"
9,IndiGo,15/10/2021,Patna,Chennai,16:30,23:35,1 stop via New Delhi,"₹ 7,512"


Lenght :  28
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=PAT-MAA-17/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 19
Time Count in Scraping  = 38
City Count in Scraping  = 38
spanFlightCost Count in Scraping  = 19
pFlightStops Count in Scraping  = 19
Transaform Function Running.... 


...

Lenght :  2058
 
--------------
origin = jknaknnaknja
destin = PAT

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=jknaknnaknja-PAT-15/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
EXCEPTION IN (<ipython-input-3-e63b93705f05>, LINE 25 "element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))"): Message: 

15/10/2021  scraping complete
 
origin = jknaknnaknja
destin = jknaknnaknja

-----Skiped-----
 
origin = jknaknnaknja
destin = MAA

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=jknaknnaknja-MAA-15/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
EXCEPTION IN (<ipython-input-3-e63b93705f05>, LINE 25 "element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))"): Message: 

15/10/2021  scraping complete
 
origin = jknaknnaknja
destin = LKU

Requesting URL: https://www.makemytrip.com/fligh

,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Go First,15/10/2021,Patna,Chennai,16:55,23:15,1 stop via Mumbai,"₹ 7,511"
2,IndiGo,15/10/2021,Patna,Chennai,08:40,16:10,1 stop via Bengaluru,"₹ 7,512"
3,IndiGo,15/10/2021,Patna,Chennai,11:20,17:30,1 stop via Mumbai,"₹ 7,512"
4,IndiGo,15/10/2021,Patna,Chennai,11:35,17:55,1 stop via Hyderabad,"₹ 7,512"
...,...,...,...,...,...,...,...,...
2075,IndiGo,15/10/2021,Chennai,Patna,12:20,19:10,1 stop via New Delhi,"₹ 8,644"
2076,Air India,15/10/2021,Chennai,Patna,07:00,14:10,1 stop via Kolkata,"₹ 8,928"
2077,"Air India, IndiGo",15/10/2021,Chennai,Patna,07:00,12:50,1 stop via Kolkata,"₹ 9,852"
2078,"AirAsia, Air India",15/10/2021,Chennai,Patna,07:50,14:10,1 stop via Kolkata,"₹ 9,852"


Lenght :  2080
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=MAA-PAT-16/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 17
Time Count in Scraping  = 34
City Count in Scraping  = 34
spanFlightCost Count in Scraping  = 17
pFlightStops Count in Scraping  = 17
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Go First,15/10/2021,Patna,Chennai,16:55,23:15,1 stop via Mumbai,"₹ 7,511"
2,IndiGo,15/10/2021,Patna,Chennai,08:40,16:10,1 stop via Bengaluru,"₹ 7,512"
3,IndiGo,15/10/2021,Patna,Chennai,11:20,17:30,1 stop via Mumbai,"₹ 7,512"
4,IndiGo,15/10/2021,Patna,Chennai,11:35,17:55,1 stop via Hyderabad,"₹ 7,512"
...,...,...,...,...,...,...,...,...
2092,IndiGo,16/10/2021,Chennai,Patna,06:20,14:20,1 stop via New Delhi,"₹ 7,998"
2093,IndiGo,16/10/2021,Chennai,Patna,10:00,15:55,1 stop via New Delhi,"₹ 7,998"
2094,Air India,16/10/2021,Chennai,Patna,06:10,15:20,1 stop via New Delhi,"₹ 9,141"
2095,Vistara,16/10/2021,Chennai,Patna,10:50,19:50,1 stop via New Delhi,"₹ 9,949"


Lenght :  2097
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=MAA-PAT-17/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
